# Tutorial on the IPython handler with basf2

Hello and welcome to this tutorial in the usage of IPython with basf2. To dive right into it I assume you are already familiar with the Belle Software Analysis Framework 2 (basf2) and also a bit with python itself.
If that is not the case now would be the best time to find some more information on basf2 on the [internal twiki](https://confluence.desy.de/display/BI/Software+Basf2SoftwarePortal) and on python everywhere in the [web](https://docs.python.org/2/tutorial/). But do not worry: we will only need some basic knowledge here.

As you have already opened this notebook in a way I assume you know how to start an IPython server (docker user do not need to think about that). If not:
* Assert you have IPython installed on your virtualenv of basf2:


    setuprel and stuff
    pip3 install "ipython[all]"
    
* Start the ipython server
   
   
    ipython3 notebook
    
This will automatically start your browser pointing to localhost:8888 - the default place for the notebook server. You can change this and many other options with the config files of [IPython](https://ipython.org/ipython-doc/dev/config/intro.html). You can get more information on IPython on the [website](https://ipython.org/) of the project.

For the plots in this tutorial we will need pandas, matplotlib, numpy and seaborn. You can install all of them with 

`pip3 install matplotlib numpy pandas seaborn` 

or you go on with the tutorial without the plots.

For doing basf2 calculations in this notebook in a convenient way, we use the `handler` class of the `ipython_tools` package (for more information and feedback [send me a mail](mailto:nils.braun@student.kit.edu)). If you use the docker image, the package is already loaded. For all others execute the following cell by pressing *Shift-Enter* with the cell activated.

In [ ]:
from ipython_tools import handler
handler.style()

You can do anything in IPython that you can do with python also - for example you could create a basf2 path and process as you would do in your normal steering file. The `ipython_tools` package is just for convenience (and lots of addon features). 

You can edit the cells by clicking into them or navigating to the cells with your arrow keys and pressing *Enter*. You can change the type of the cells from `Code` to `Markdown` or `Raw` with the options in the menu bar or by pressing the corresponding hotkeys *Y*, *M* and *R*. Checkout the hotkeys by pressing *Escape* and then *h* and look into the manual of IPython to see how to edit and decorate your notebooks.

Later, we will create some plots. To show them inline, do a

In [ ]:
%matplotlib inline

(docker users do not need this also). Now we can start with the examples.

## First example

As a first example, we will create 10 generic BBbar events and call the whole simulation/reconstruction chain on them. We will then output these simulated events into a root file. This module path is just for showing you the capabilities of the `ipython_tools.handler` - and is therefore very simple.

As a starting point, we declare our path as we would have done in the steering file:

In [ ]:
import basf2
from modularAnalysis import generateY4S
from simulation import add_simulation
from beamparameters import add_beamparameters
from reconstruction import add_reconstruction, add_mdst_output

main_path = basf2.create_path()
add_beamparameters(main_path, "Y4S")
generateY4S(10, path=main_path)
add_simulation(path=main_path)
add_reconstruction(path=main_path)

add_mdst_output(filename="output_simulation_reconstruction_10.root", path=main_path)

We end up with a basf2.path object that we normally would use to call basf2.process on it. This would block the entire IPython kernel until the calculation is stopped. Also, every error in your basf2 calculation will result in a broken kernel in IPython. Worse: you can not access the log files or stop the calculation once running. We better use the `ipython_tools` for this. So use the process function of the handler:

In [ ]:
calculation = handler.process(main_path)
calculation

Nothing has happened so far. The calculation is just a container for one (or more as you will see later) paths that can be processed. Before starting the calculation we want to investigate some of the methods of the calculation.

For example, we can investigate our constructed path a bit further (this make take about 2-3s as the path is quite large):

In [ ]:
calculation.show_path()

You end up with a clickable representation of the path with all its modules with their parameters in black and default parameters in gray (You will notice that there are more modules in the path then you added. These are needed for the IPython convenience functions and do not have an impact on the performance). You can also get some information on the status of the calculation:

In [ ]:
print(calculation.get_status())
print(calculation.is_running())

More of the methods of the calculation are more meaningful if the calculation is actually running (or finished already). So lets start the calculation!

In [ ]:
calculation.start()

Nothing happened, you might think. Not quite correct. The calculation is running in the background. You can go on with other things in your notebook (like creating the analysis script while the data creation is running). You can even close your browser and switch to another machine. As long as the IPython kernel (and the IPython notebook server) is still running, you can reconnect to this notebook every time. The calculation will go on while you are absent.

You can check the status of the calculation with:

In [ ]:
print(calculation.get_status())

.. which should show 'running' or 'finished' (if you have a really fast machine). You can also show a nice bar showing the progress of the calculation (do not mind if there seems to be no progress, the init of the path lasts that long because the whole geometry has to be loaded first):

In [ ]:
calculation.wait_for_end()

As the name of the function suggests this call blocks the whole notebook execution until the calculation is finished or has failed (but not breaks the kernel or other nasty things). See the next section for ways to stop a calculation or abort an already running one. 

After the calculation is finished, you can double check the status (the green bar suggests already a positive result):

In [ ]:
print(calculation.get_status())
print(calculation.is_running())
print(calculation.is_finished())
print(calculation.has_failed())

Great! We can now use the other methods of the calculation object to inspect the processing of the path. First, we want to check which modules perform best:

In [ ]:
calculation.show_statistics()

Or we can have a look into the DataStore:

In [ ]:
calculation.show_collections()

This shows you the content of the data store with the numbers of entries in the StoreArrays every 1000 events (or if you read from file every 10th part of the whole file content). 

These are the really basic features. But there is more to discover. See the next chapters for advanced concepts.

### Stop a calculation and handle errors

Ups! Started a calculation with wrong values? No problem. Just call on a running calculation:

In [ ]:
calculation.stop()

But what if you have called `wait_for_output` and the notebook is blocked? You can use the Stop-Button in the toolbar or Kernel->Interrupt in the menu to interrupt the `wait_for_output` command (not the calculation! It is running in the background, remember?). Than you can use the `calculation.stop()` as above.

But what if your calculation failed? You can get much information from the log output. You can inspect it with:

In [ ]:
calculation.show_log()

Every error, bug trace or exception should be catched by the log. As the method returns a normal python string, you can also use the python-buildin methods for searching or regexing the output.

For more advanced users: The calculation running in the background writes its output to a temporary file in your `/tmp` folder. Ths file gets read into a variable and deleted when you first call `calculation.get_log()`. If you do not call the function, the log file will never be deleted - unless more than 20 log files have been created. You can still delete the files manually if you want, but this should not affect your disk space usage much. Your linux OS deletes those files typically at midnight or on the next reboot.

## Second example - the queue and first plots

Except for some nice features like the colored status bar, the interactive collections and path viewer, the basf2 log in python and the possibility to let the calculation run in the background, we did not use the big benefit of IPython until now: the interactivity. We will exploit that feature when doing plots of some variables now.

The normal workflow for doing analysis plots is: 
* Write a script which creates/reads the events with the basf2 module framework and outputs some data into a root file.
* Read in the created output file name with another script and create some plots into a png/pdf file.
* View these created plots.

Thes workflow includes 2 script files and running some externals programs (at least the image viewer + some editor for the scripts). If you want to change something you have to take care to change it in both files in sync. Also it is easy to get confused with file names between both scripts.

With IPython you can solve those problems as you will see in the following:

In this second example we will use MC data from a file which was already created. Of course you can use IPython as well for creating such files, but the simulation for many events lasts very long. Therefore, I provide an input file for you. It is filled with 1000 pregenerated $B \to J/\psi\ K_s$ events that we will use for further processing. We will perform a typical analyis in this data sample:

In [ ]:
import basf2
from modularAnalysis import *
from variables.utils import create_aliases_for_selected
file_name = "http://ekpwww.physik.uni-karlsruhe.de/~nbraun/B2JpsiKs_mu_BGx0_r1000.root"
filelistSIG = [file_name]

main_path = create_path()
queue = handler.create_queue()

inputMdstList("default", filelist=filelistSIG, path=main_path)

fillParticleList('pi+:loose', 'piid > 0.01', path=main_path)
fillParticleList('mu+:loose', 'muid > 0.01', path=main_path)

# reconstruct Ks -> pi+ pi- decay
# keep only candidates with 0.4 < M(pipi) < 0.6 GeV
reconstructDecay('K_S0:pipi -> pi+:loose pi-:loose', '0.4 < M < 0.6', path=main_path)

# reconstruct J/psi -> mu+ mu- decay
# keep only candidates with 3.0 < M(mumu) < 3.2 GeV
reconstructDecay('J/psi:mumu -> mu+:loose mu-:loose', '3.0 < M < 3.2', path=main_path)

# reconstruct B0 -> J/psi Ks decay
# keep only candidates with 5.2 < M(J/PsiKs) < 5.4 GeV
reconstructDecay('B0:jspiks -> J/psi:mumu K_S0:pipi', '5.2 < M < 5.4', path=main_path)

# perform B0 kinematic vertex fit using only the mu+ mu-
# keep candidates only passing C.L. value of the fit > 0.0 (no cut)
vertexRave('B0:jspiks', 0.0, 'B0 -> [J/psi -> ^mu+ ^mu-] K_S0', path=main_path)

# build the rest of the event associated to the B0
buildRestOfEvent('B0:jspiks', path=main_path)

# perform MC matching (MC truth asociation). Always before TagV
matchMCTruth('B0:jspiks', path=main_path)

# calculate the Tag Vertex and Delta t (in ps)
# breco: type of MC association.
TagV('B0:jspiks', 'breco', path=main_path)

# create and fill flat Ntuple with MCTruth, kinematic information
interesting_variables = ['px', 'py', 'pz', 'E', 'isSignal', 'mcPDG']
variables = create_aliases_for_selected(
    'B0 -> [J/psi -> ^mu+ ^mu-] [K_S0 -> ^pi+ ^pi-]', interesting_variables)

# write out the flat ntuples
queue.put('output_file_name', 'B2A410-TagVertex.root')
variablesToNtuple('B0:jspiks', variables, filename='B2A410-TagVertex.root', path=main_path)

Everything is more or less the same as in one of the examples in the analysis package of basf2. Except for the created queue. A Basf2Queue is the basic communication channel between you and your calculation. Every calculation has a distinct queue it uses for writing calculation-dependent results or parameters onto. The IPython handler itself uses it throughout the whole processing and you can use it to.

To write onto the queue just call its `put` method whenever you like (before the start, after the start or even after the end). Hand in a name for better reference and the value. If you hand in the same name twice the older value will be overriten. You can hand in every pickable object but please keep in mind that these data has to be send from process to process - so keep the data as small as possible. One of its main features is to save all calulation-related file names (like the output file name in the example above) to have everything related to this calculation stored in one place. 

Warning: The queue is not a replacement for the `DataStore`! Plese use it only for calculation-related thinks like filenames which you want to preserve after the calculation also and keep it together with the process.

We will create a calculation and start it like before. This time we hand in the queue as a parameter also.

In [ ]:
calculation = handler.process(main_path, max_event=1000, result_queue=queue)
calculation.start()
calculation.wait_for_end()

After the calculation has finished, you can access its queue values. To get a list of all keys, do a:

In [ ]:
calculation.get_keys()

As you can see, there is the output file name we put there ourselves and two other values the IPython handler uses. You do not need to care about them. You can get the value for a specific key with:

In [ ]:
calculation.get("output_file_name")

Everytime you do an analysis of the data calculated with this path, you have every information stored in the same spot: the path, the log, the parameters and results and the statistics of the process (like you have seen before). We can use this information now to draw some plots of the variables. We use `uproot` to open the variables tree in the root file:

In [ ]:
import uproot
dataframe = uproot.open(calculation.get("output_file_name"))["variables"].arrays(library="pd")
dataframe.head()

Every numerical column in the root file is written into a `pandas.DataFrame`. We can use the pandas own functions for plotting. Maybe you are interested in the reconstructed vertex position in regard to the correct one?

In [ ]:
(dataframe.B0_X - dataframe.B0_TruthX).hist(bins=50, range=[-0.02, 0.02])

If you want to get familiar with the plotting features pandas have, please see the [website](http://pandas.pydata.org/) of the project.

## Using the queue

In the example above we used the queue to store related information before we processed the path. We could have - in principle - stored the file name in another variable also and reuse this variable when doing the plots (you need to be careful to not mix the variables up, but it works). But there is also another use case for the queue you can not do with pure variables: accessing data within the event run. As the path is processed in another process (because it needs to run in the background), you can not access its data. You have to use the queue for this. For example, you can write a module which counts the number of events passed and writes them to the queue in the end (this is obviously a very silly example because there are many other ways to access this information):

In [ ]:
class SillyModule(basf2.Module):
    # Pass in the queue
    def __init__(self, queue):
        self.queue = queue
        self.event_number = 0
        
        basf2.Module.__init__(self)
        
    # Increase the event number counter by 1 every event
    def event(self):
        self.event_number += 1
        
    # Write the event number to the queue
    def terminate(self):
        self.queue.put("event_counter", self.event_number)

We will use the path from above again and append out module to it:

In [ ]:
main_path.add_module(SillyModule(queue))
calculation = handler.process(main_path, result_queue=queue)
calculation.start()
calculation.wait_for_end()
calculation.get_keys()

Our module has written something to the queue! Lets see what:

In [ ]:
calculation.get("event_counter")

Very good. We expected that many events. You can write arbitrary things to the queue whenever you want - they just need to be pickable (which is the case for mostly every python object).

## Third example - multiprocessing of paths and more plots


Now you know how to process paths, use the queue and do some easy plots. But now we want to do some "real" science (do not expect much from that <img style="width: 20px; display: inline;" src="http://ecx.images-amazon.com/images/I/31C8iwCJ7ZL._SX300_.jpg"></img>). We want to make a study whether we can increasy the quality of the vertex fit by applying another cut on the $\chi^2$ and changing the cut on the B mass. We do this by using another feature of the IPython handler: the process_parameter_space. We will create different paths with different input variables for $\chi^2$ and $M_B$ and create a calculation out of them. We will process them all at once and create a plot with all the information in it. So lets start!

First we create a function to return our path for the given input variables:

In [ ]:
def path_creator(chi2_cut, m_band_size, queue):
    main_path = create_path()

    inputMdstList("default", filelist=filelistSIG, path=main_path)
    fillParticleList('pi+:loose', 'piid > 0.01', path=main_path)
    fillParticleList('mu+:loose', 'muid > 0.01', path=main_path)
    reconstructDecay('K_S0:pipi -> pi+:loose pi-:loose', '0.4 < M < 0.6', path=main_path)
    reconstructDecay('J/psi:mumu -> mu+:loose mu-:loose', '3.0 < M < 3.2', path=main_path)
    # We put the M in here:
    reconstructDecay('B0:jspiks -> J/psi:mumu K_S0:pipi', '%.3f < M < %.3f' % (5.3 - m_band_size/2.0, 
                                                                               5.3 + m_band_size/2.0), path=main_path)
    # we put the chi2 in here:
    vertexRave('B0:jspiks', chi2_cut, 'B0 -> [J/psi -> ^mu+ ^mu-] K_S0', path=main_path)
    buildRestOfEvent('B0:jspiks', path=main_path)
    matchMCTruth('B0:jspiks', path=main_path)
    TagV('B0:jspiks', 'breco', path=main_path)

    # create and fill flat Ntuple with MCTruth, kinematic information
    interesting_variables = ['px', 'py', 'pz', 'E', 'isSignal', 'mcPDG']
    variables = create_aliases_for_selected(
        'B0 -> [J/psi -> ^mu+ ^mu-] [K_S0 -> ^pi+ ^pi-]', interesting_variables)

    # write out the flat ntuples
    output_file_name = "B2A410-TagVertex_%.3f_%.3f.root" % (chi2_cut, m_band_size)
    queue.put('output_file_name', output_file_name)
    variablesToNtuple('B0:jspiks', variables, filename=output_file_name, path=main_path)
    
    return {"path": main_path, "max_event": 1000}

We can use this function to create paths with the given set of input variables `chi2_cut` and `m_band_size`. What we want now is the following:
* Create every combination of the parameters (so every chi2 we choose should go with every m band size).
* Create a path with these parameters and create a calculation out of it.
* Process all calculations in parallel and keep track of the information stored to each calculation and its results.

You can have all of this with the `process_parameter_space` function of the handler:

In [ ]:
calculations = handler.process_parameter_space(path_creator, chi2_cut=[0.0, 0.1], m_band_size=[0.1, 0.2])

The parameters of this funtion is a function-like object path_creator and as many keyword parameters with lists as you like. The path_creator is used to create each path - so it should return a basf2 path or None (then the calculation will not be used, but saved in the lists of calculations for reference and counting). The parameters of the path_creator can be used to control the created path. Each parameter you define in the path_creator function has to be also in the rest of the process_parameter_space function. So if you path_creator looks like:

    def path_creator(x, y, z):
        return path
        
your call to process_parameter_space should look like:

    handler.process_parameter_space(path_creator, x=[ ... ], y=[ ... ], z=[ ... ])
    
The order does not matter here, only the name. One exception is the parameter name queue. You can (but you do not need to) use this parameter in your path_creator function but not in the process_parameter_space call. It is filled by the IPython handler with the queue associated with this calculation when the path_creator is called. So you can put some additional information on the queue or use it when you create the path. Every single path gets its own queue.

The `calculations` object returned is a list of several calculations - each one like the ones before. Actually, the returned object is exactly like the Basf2Calculation before - it does not matter if you handle one of 1000 calculations. You can use the same methods on the calculation as before - except that you will get a list of outputs now, not a single one. The calculation itself acts like a list and you can iterate over it to get the single calculations.

For example, you can check the status of the calculations:

In [ ]:
calculations.get_status()

Or you can check which parameters were used to create the single calculations:

In [ ]:
calculations.get_parameters()

All functions returning values now return a list of values. All function returning a widget (like show_collections, show_statistics etc.) now return a tab view of widgets.
There is also the possibility to access the information of only a single calculation in the list. Just pass a number with a function of the Basf2Calculation:

In [ ]:
calculations.get_parameters(2)

Of course, you can start, stop and wait for the calculation as before. Run the calculations now. As all run in parallel, the execution time should be as small as with one single calculation (assuming you have 4 cores).

In [ ]:
calculations.start()
calculations.wait_for_end()

Also the queue works as expected. You end up with a list instead of a single value:

In [ ]:
calculations.get("output_file_name")

We can now use the results to create a pandas dataframe with all the results. First we import all the data, then we concat it to a big dataframe.

In [ ]:
import pandas as pd
import numpy as np

dataframes_with_parameters = [(c.get_parameters(), read_root(c.get("output_file_name"))) for c in calculations]

for parameter, dataframe in dataframes_with_parameters:
    dataframe["chi2_cut"] = parameter["chi2_cut"]
    dataframe["m_band_size"] = parameter["m_band_size"]
    
result_dataframe = pd.concat([dataframe for parameter, dataframe in dataframes_with_parameters], ignore_index=True)

result_dataframe["reco_difference"] = np.sqrt((result_dataframe.B0_X - result_dataframe.B0_TruthX)**2 + 
                                              (result_dataframe.B0_Y - result_dataframe.B0_TruthY)**2 + 
                                              (result_dataframe.B0_Z - result_dataframe.B0_TruthZ)**2)

We can now see of the change in the chi2 cut and the mass side bands had an impact on the vertex position reconstruction (this is more or less advanced pandas code. We could have done this also by building a loop over all results, but this is faster):

In [ ]:
import seaborn as sb
results = result_dataframe.groupby(["chi2_cut"])['reco_difference'].agg({
                                                           'number of reconstructed vertices' : len,
                                                           'median of reconstruction error' : np.median,
                                                           'std of reconstruction error' : np.std})
results['number of reconstructed vertices'].plot()

As you can see the number of reconstructed vertices decreases with the $\chi^2$-cut, which is expected. You can analyse the other variables in the data frame `results` if you like, but for a real analysis the datasample is too small. You can use a bigger datasample if you like and redo this analysis by just changing the input file name and recalculating the cells.

## After the calculation

Have found something new and special? Want to share your thoughts and analysis with other people? No problems with IPython. You can send your notebook as a file to other people. As you have everything in one place it is easy for them to reado your calculation. As the plots and numbers are already saved into the notebook they even do not need to redo your calculation - they just need to look onto the code and then onto your results and plots and can easily retrace your thoughts. By adding comments with markdown (like this one) and even html tags you can include text, images, videos and so on.

If you do not want to send your notebook as an IPython file (because maybe the other person does not have Ipython), you can export your notebook into various formats like pdf, html or md. See the `File` menu for this. You can even create a slideshow (see [here](http://ipython.org/ipython-doc/2/notebook/nbconvert.html)). 

If doing a bug report or tracing a strang behaviour not only the code and the results but also information on the used release and external number is needed. You can access these values with:

In [ ]:
handler.information

Just include this line in your notebook and everyone will know what to do to reproduce your bug.

## Expert knowledge

As you may have noticed, we did not set a random seed in any of our calculations. This is only needed for simulation as no reconstruction or analysis code should depend on random numbers in is therefore a more or less advanced topic. Of course, you can set the random seed of the calculations with the IPython handler also, but you have to take into account that the calculation is done in another process, so just doing a `basf2.set_random_seed` will not work as this sets the random number in the process of your notebook but not of the process calculating your path. If you want to set the random seed, pass it to the `process` function:

    handler.process(path, [result_queue=queue,] random_seed=seed)
    
or as a list to the `process_parameter_space`:

    handler.process_parameter_space(path, param1=[..], ..., random_seed=seed)

Thank you for doing this tutorial. If you have still questions, bugs or feature requests, do not hesitate to [contact me](mailto:nils.braun@student.kit.edu).